In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

estadisticas = []
T = [1,2,3,4,5,6,7,8]
title_problems = ["Rosenbrock","Sinoidal","Celda"]
problems_number = { "Rosenbrock" : 0, "Sinoidal": 1, "Celda": 2 }

In [4]:
for e in range(0,3):
    file = open(f"./outputs/{e+1}_S_1.out", "r")
    secS = int(file.readlines()[2]) / 1000
    estadisticas.append( [title_problems[e], "Secuencial", 1, secS, 1, 1] )

for e in range(0,3):
    file = open(f"./outputs/{e+1}_S_1.out", "r")
    secS = int(file.readlines()[2]) / 1000
    for t in T:
        file = open(f"./outputs/{e+1}_P_{t}.out", "r")
        secP = int(file.readlines()[2]) / 1000
        speedup = secS/secP
        efficiency = speedup / t
        estadisticas.append( [title_problems[e], "Paralelo", t, secP, speedup, efficiency] )

In [5]:
import itertools
import numpy as np

bounds = np.array(
    [
        [
            [ 0,2 ],
            [ 0,2 ]
        ],
        [
            [ -2, 2 ],
            [ -2, 2 ]
        ]
    ]
)

# [
# 	[ 0.0, 0.0, 0.0, 0.0, 0.0 ],
# 	[ 0.5, 0.2, 1.0, 1.14, 30.0 ]
# ],

def f_rosenbrock(x):
	return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def f_sinoidal(x):
	return x[0]**2 + x[1]**2 + 3.0 * np.sqrt(np.sin(5 * x[0])**2 + np.sin(5 * x[1])**2) + 0.1

def gplot(e, f):
	file = open(f"./outputs/{e+1}_S_1.out", "r")
	x = np.array(file.readlines()[0].split(" "))
	x = np.array(x[:-1], dtype = float)
	t = 100
	xx = np.linspace(bounds[e,0,0], bounds[e,0,1], t)
	yy = np.linspace(bounds[e,1,0], bounds[e,1,1], t)
	fig = go.Figure()
	fig.add_surface(
		x = xx,
		y = yy,
		z = np.array([f(np.array([x_i,y_i])) for y_i, x_i in itertools.product(yy,xx)]).reshape(t,t),
		name = 'f'
	)
	fig.add_trace(
		go.Scatter3d(
			x = [x[0]],
			y = [x[1]],
			z = [f(x)],
			name = "Serial"
		)
	)
	fig.update_layout(
			template="plotly_dark",
			title=title_problems[e],
			width=1000,
			height=1000
	)
	for t in T:
		file = open(f"./outputs/{e+1}_P_{t}.out", "r")
		x = np.array(file.readlines()[0].split(" "))
		x = np.array(x[:-1], dtype = float)
		fig.add_trace(
		go.Scatter3d(
			x = [x[0]],
			y = [x[1]],
			z = [f(x)],
			name = "Paralelo t = " + str(t)
		)
	)
	fig.show()

gplot(0, f_rosenbrock)
gplot(1, f_sinoidal)

In [35]:
from math import log2, log

bounds = [0.001, 3.5]

def V(x, i):
	return x[0] - x[1] * log2(i) - x[2] * i + x[3] * log(1 - x[4] * i)

VD = np.array(
	[
		[ 0.005019, 0.010038, 0.015, 0.019962, 0.0258365, 0.030057, 0.0317681, 0.0319962, 0.0330228, 0.0337072 ],
		[ 0.69844, 0.6125, 0.58281, 0.5375, 0.47344, 0.36563, 0.31094, 0.2625, 0.2, 0.11094 ]
	]
)
VD = VD.T
VD = np.sort(VD)
VD = VD.T

density = VD[0]
voltage = VD[1]

def f_celda(x):
    ans = 0.0
    for d, v in zip(density, voltage):
        ans += (v - V(x, d))**2
    return ans

def gplot(e, f):
	file = open(f"./outputs/{e+1}_S_1.out", "r")
	x = np.array(file.readlines()[0].split(" "))
	x = np.array(x[:-1], dtype = float)
	t = 1000
	xx = np.linspace(density[0], density[-1], t)
	fig = go.Figure()
	fig.add_trace(
		go.Scatter(
			x = density,
			y = voltage,
			name = title_problems[e],
			mode = "markers"
		)
	)
	fig.add_trace(
		go.Scatter(
			x = xx,
			y = [V(x, i) for i in xx],
			name = "Serial"
		)
	)
	fig.update_layout(
		template="plotly_dark",
		title=title_problems[e],
		width=1000,
		height=1000
	)
	for t in T:
		file = open(f"./outputs/{e+1}_P_{t}.out", "r")
		x = np.array(file.readlines()[0].split(" "))
		x = np.array(x[:-1], dtype = float)
		xx = np.linspace(density[0], density[-1], t)
		fig.add_trace(
			go.Scatter(
				x = xx,
				y = [V(x, i) for i in xx],
				name = "Paralelo t = " + str(t)
			)
		)
	fig.show()

gplot(2, f_celda)

In [6]:
df = pd.DataFrame(estadisticas, columns=["Problema", "Modo de Ejecucion", "Hilos", "Tiempo(ms)", "Speedup", "Efficiency"])
df.head(-1)

,Problema,Modo de Ejecucion,Hilos,Tiempo(ms),Speedup,Efficiency
0,Sinoidal,Secuencial,1,34922.978,1.000000,1.000000
1,Celda,Secuencial,1,5557.640,1.000000,1.000000
2,Rosenbrock,Secuencial,1,34922.978,1.000000,1.000000
3,Sinoidal,Secuencial,1,5557.640,1.000000,1.000000
4,Celda,Secuencial,1,18667.073,1.000000,1.000000
5,Rosenbrock,Paralelo,1,35128.925,0.994137,0.994137
6,Rosenbrock,Paralelo,2,29698.301,1.175925,0.587963
7,Rosenbrock,Paralelo,3,25838.263,1.351599,0.450533
8,Rosenbrock,Paralelo,4,24880.271,1.403641,0.350910
9,Rosenbrock,Paralelo,5,23939.791,1.458784,0.291757


In [27]:
medias = df.groupby(["Problema", "Modo de Ejecucion", "Hilos"], as_index=False).mean().round(5)
colors = ["#A9D6E5","#89C2D9","#61A5C2","#468FAF","#2C7DA0","#277596"]
table_colors = [[colors[(problems_number[p["Problema"]]-1)*2+(1 if p["Modo de Ejecucion"] == "Secuencial" else 0)] for _, p in medias[["Problema", "Modo de Ejecucion"]].iterrows()]*4]

fig = go.Figure(
    data = [
        go.Table(
            header=dict(
                        values=list(medias.columns),
                        line_color ='#012A4A',
                        fill_color ='#012A4A',
                        font_color = 'white',
                        align ='center'
                        ),
            cells=dict(
                        values = [medias[col] for col in medias.columns],
                        line_color = table_colors,
                        fill_color = table_colors,
                        font_color = "#444c57",
                        align = 'center'
                        ),
            
        )
    ],
    layout = dict(height=1300, width = 800),
)

fig.update_layout(template = "plotly_dark")
fig.show()

In [28]:
for i in range(0,3):
    medias_i = medias.loc[medias["Problema"] == title_problems[i]].drop("Problema", axis = 1)
    fig = go.Figure(
        data = [
            go.Table(
                header=dict(
                            values=list(medias_i.columns),
                            line_color ='#012A4A',
                            fill_color ='#012A4A',
                            font_color = 'white',
                            align ='center'
                            ),
                cells=dict(
                            values = [medias_i[col] for col in medias_i.columns],
                            line_color = colors[(i-1)*2],
                            fill_color = colors[(i-1)*2],
                            font_color = "#444c57",
                            align = 'center'
                            ),
                
            )
        ],
        layout = dict(height=800, width = 800),
    )

    fig.update_layout(template = "plotly_dark")
    fig.show()

In [24]:
import numpy as np

problema = "Rosenbrock"
problemaAct = medias.loc[(medias["Problema"] == problema) & (medias["Modo de Ejecucion"] == "Paralelo")]

fig = make_subplots(rows=1, cols=2, subplot_titles=["Speedup","Efficiency"])

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Speedup"].values,
      name = "Paralelo"
  ),
  row = 1,
  col = 1
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Hilos"].values,
      name = "Ideal"
  ),
  row = 1,
  col = 1
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Efficiency"].values,
      name = "Paralelo"
  ),
  row = 1,
  col = 2
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = np.repeat(1, len(problemaAct["Hilos"].values)),
      name = "Ideal"
  ),
  row = 1,
  col = 2
)

fig.update_layout(template = "plotly_dark")
fig.show()

In [25]:
problema = "Sinoidal"
problemaAct = medias.loc[(medias["Problema"] == problema) & (medias["Modo de Ejecucion"] == "Paralelo")]

fig = make_subplots(rows=1, cols=2, subplot_titles=["Speedup","Efficiency"])

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Speedup"].values,
      name = "Paralelo"
  ),
  row = 1,
  col = 1
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Hilos"].values,
      name = "Ideal"
  ),
  row = 1,
  col = 1
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Efficiency"].values,
      name = "Paralelo"
  ),
  row = 1,
  col = 2
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = np.repeat(1, len(problemaAct["Hilos"].values)),
      name = "Ideal"
  ),
  row = 1,
  col = 2
)

fig.update_layout(template = "plotly_dark")
fig.show()

In [26]:
problema = "Celda"
problemaAct = medias.loc[(medias["Problema"] == problema) & (medias["Modo de Ejecucion"] == "Paralelo")]

fig = make_subplots(rows=1, cols=2, subplot_titles=["Speedup","Efficiency"])

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Speedup"].values,
      name = "Paralelo"
  ),
  row = 1,
  col = 1
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Hilos"].values,
      name = "Ideal"
  ),
  row = 1,
  col = 1
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = problemaAct["Efficiency"].values,
      name = "Paralelo"
  ),
  row = 1,
  col = 2
)

fig.add_trace(
  go.Scatter(
      x = problemaAct["Hilos"].values,
      y = np.repeat(1, len(problemaAct["Hilos"].values)),
      name = "Ideal"
  ),
  row = 1,
  col = 2
)

fig.update_layout(template = "plotly_dark")
fig.show()